# Junior Job Data Cleaning

This notebook is just to clean data that will go into Tableau. We have text data so we need something like Python to do this. The end goal is to look at different types of jobs and filter by them to see what trends occur in junior level positions.

In [139]:
import numpy as np
import pandas as pd
import string
import re
import ast
import nltk
from nltk.corpus import stopwords
from collections import Counter
from collections import defaultdict

In [140]:
orig_df = pd.read_csv('Jobs_Data.csv')

#want to keep original just in case, since a lot of changes will occur
df = orig_df.copy()

df.head()

,company,description,focus,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience
0,Novovu,[b]We are looking for a talented frontend web ...,Frontend Developer,"Frontend Developer (HTML, CSS, and JS) - Remote",https://lensa.com/frontend-developer-html-css-...,"Fort Washington, PA, USA","Jul 14, 2023 8:00 am",100% Remote,$,[],Full-time,0-1 year
1,Coalition Technologies,[heading2]WHY YOU SHOULD APPLY:[/heading2]\r\n...,Frontend Developer,Front End Developer,https://www.virtualvocations.com/job/remote-fr...,NaN,"Jul 14, 2023 8:01 am",100% Remote,$,"[15,35]",Full-time,0-1 year
2,Get It Recruit - Information Technology,[heading2]Job Description[/heading2]\r\n\r\nAs...,Backend Developer,Software Developer - Remote | WFH,https://www.linkedin.com/jobs/view/software-de...,"King George, VA 22485, USA","Jul 14, 2023 8:03 am",100% In-Office,$,"[72000,182000]",Full-time,0-1 year
3,SiLo,"[b]Based in Downtown Nashville, Simple Logisti...",Fullstack Developer,Junior .NET Full Stack Developer,https://jobs.wjhl.com/jobs/junior-.net-full-st...,"Nashville, TN, USA","Jul 14, 2023 8:04 am",100% In-Office,$,[],Full-time,0-1 year
4,eDiligence,[center][size=4][b]Entry Level Full Stack Deve...,Fullstack Developer,Entry Level Software Developer,https://jooble.org/jdp/-2571818786162603564/En...,"Los Angeles, CA, USA","Jul 14, 2023 8:05 am",Hybrid Work,$,[],Full-time,0-1 year


In [141]:
df.describe()

,company,description,focus,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience
count,1426,1422,1426,1426,1426,1057,1426,1426,1424,1426,1426,1426
unique,943,1202,85,845,1244,367,895,3,1,261,3,4
top,Guidehouse,[b]Position Summary...[/b]\r\n\r\nWhat you'll ...,Data Analyst,Data Analyst,https://www.ziprecruiter.com/c/Reesby/Job/UI-U...,United States,"Aug 1, 2023 8:00 pm",Hybrid Work,$,[],Full-time,0-1 year
freq,27,15,210,48,10,54,17,595,1424,1032,1353,879


In [142]:
countCompany = Counter(df['company'])

max(countCompany.values())

27

Want to count most frequent company since if it is low then can conclude that this feature is probably not useful.

In [143]:
def get_key_by_value(dictionary, target_value):
    for key, value in dictionary.items():
        if value == target_value:
            return key
    return None

In [144]:
get_key_by_value(countCompany, 27)

'Guidehouse'

Most common company is Guidehouse with only 27 occurences, so this is not a significant feature to look at, especially if broken down by job title.

Get the hour of the job posting.

In [145]:
df['postingHour'] = df['postedDate'].apply(lambda x: x[-7] + ' ' + x[-2:])

In [146]:
Counter(df['postingHour'])

Counter({'8 am': 1094,
         '9 am': 149,
         '0 am': 66,
         '5 pm': 10,
         '4 am': 4,
         '1 pm': 17,
         '6 am': 7,
         '2 am': 3,
         '3 pm': 6,
         '1 am': 5,
         '7 pm': 6,
         '8 pm': 33,
         '0 pm': 3,
         '7 am': 3,
         '6 pm': 3,
         '5 am': 2,
         '2 pm': 6,
         '3 am': 3,
         '9 pm': 2,
         '4 pm': 4})

In [147]:
df.head()

,company,description,focus,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience,postingHour
0,Novovu,[b]We are looking for a talented frontend web ...,Frontend Developer,"Frontend Developer (HTML, CSS, and JS) - Remote",https://lensa.com/frontend-developer-html-css-...,"Fort Washington, PA, USA","Jul 14, 2023 8:00 am",100% Remote,$,[],Full-time,0-1 year,8 am
1,Coalition Technologies,[heading2]WHY YOU SHOULD APPLY:[/heading2]\r\n...,Frontend Developer,Front End Developer,https://www.virtualvocations.com/job/remote-fr...,NaN,"Jul 14, 2023 8:01 am",100% Remote,$,"[15,35]",Full-time,0-1 year,8 am
2,Get It Recruit - Information Technology,[heading2]Job Description[/heading2]\r\n\r\nAs...,Backend Developer,Software Developer - Remote | WFH,https://www.linkedin.com/jobs/view/software-de...,"King George, VA 22485, USA","Jul 14, 2023 8:03 am",100% In-Office,$,"[72000,182000]",Full-time,0-1 year,8 am
3,SiLo,"[b]Based in Downtown Nashville, Simple Logisti...",Fullstack Developer,Junior .NET Full Stack Developer,https://jobs.wjhl.com/jobs/junior-.net-full-st...,"Nashville, TN, USA","Jul 14, 2023 8:04 am",100% In-Office,$,[],Full-time,0-1 year,8 am
4,eDiligence,[center][size=4][b]Entry Level Full Stack Deve...,Fullstack Developer,Entry Level Software Developer,https://jooble.org/jdp/-2571818786162603564/En...,"Los Angeles, CA, USA","Jul 14, 2023 8:05 am",Hybrid Work,$,[],Full-time,0-1 year,8 am


Office location, years of experience, and type features need no cleaning:

In [148]:
Counter(df['remote'])

Counter({'100% Remote': 333, '100% In-Office': 498, 'Hybrid Work': 595})

In [149]:
Counter(df['type'])

Counter({'Full-time': 1353, 'Part-time': 38, 'Internship': 35})

In [150]:
Counter(df['yearsOfExperience'])

Counter({'0-1 year': 879, '1-3 years': 545, '5+ years': 1, '3-5 years': 1})

Almost all salaries are in $, and two are nan, so we can ignore this feature.

In [151]:
Counter(df['salaryCurrency'])

Counter({'$': 1424, nan: 2})

Want to split the salary range into two features.

In [152]:
#since the original feature is a string
df['salaryRange'] = df['salaryRange'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and x != '[]' else np.nan)

In [153]:
df['minSalary'] = df['salaryRange'].apply(lambda x: x[0] if x is not np.nan else np.nan)
df['maxSalary'] = df['salaryRange'].apply(lambda x: x[1] if x is not np.nan else np.nan)

In [154]:
df.head(7)

,company,description,focus,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience,postingHour,minSalary,maxSalary
0,Novovu,[b]We are looking for a talented frontend web ...,Frontend Developer,"Frontend Developer (HTML, CSS, and JS) - Remote",https://lensa.com/frontend-developer-html-css-...,"Fort Washington, PA, USA","Jul 14, 2023 8:00 am",100% Remote,$,NaN,Full-time,0-1 year,8 am,NaN,NaN
1,Coalition Technologies,[heading2]WHY YOU SHOULD APPLY:[/heading2]\r\n...,Frontend Developer,Front End Developer,https://www.virtualvocations.com/job/remote-fr...,NaN,"Jul 14, 2023 8:01 am",100% Remote,$,"[15, 35]",Full-time,0-1 year,8 am,15.0,35.0
2,Get It Recruit - Information Technology,[heading2]Job Description[/heading2]\r\n\r\nAs...,Backend Developer,Software Developer - Remote | WFH,https://www.linkedin.com/jobs/view/software-de...,"King George, VA 22485, USA","Jul 14, 2023 8:03 am",100% In-Office,$,"[72000, 182000]",Full-time,0-1 year,8 am,72000.0,182000.0
3,SiLo,"[b]Based in Downtown Nashville, Simple Logisti...",Fullstack Developer,Junior .NET Full Stack Developer,https://jobs.wjhl.com/jobs/junior-.net-full-st...,"Nashville, TN, USA","Jul 14, 2023 8:04 am",100% In-Office,$,NaN,Full-time,0-1 year,8 am,NaN,NaN
4,eDiligence,[center][size=4][b]Entry Level Full Stack Deve...,Fullstack Developer,Entry Level Software Developer,https://jooble.org/jdp/-2571818786162603564/En...,"Los Angeles, CA, USA","Jul 14, 2023 8:05 am",Hybrid Work,$,NaN,Full-time,0-1 year,8 am,NaN,NaN
5,Leidos Holding,[center][size=5]Description[/size][/center]\r\...,Fullstack Developer,Junior Full Stack Developer,https://clearedcareers.com/job/319461/junior-f...,"Colorado Springs, CO, USA","Jul 14, 2023 8:06 am",Hybrid Work,$,"[53300, 110700]",Full-time,0-1 year,8 am,53300.0,110700.0
6,BigBear.ai,[heading2]Overview[/heading2]\r\n\r\nBigBear.a...,Fullstack Developer,Jr. Full Stack Developer,https://getwork.com/details/bcdec3380ac6383c52...,"Washington, MA, USA","Jul 14, 2023 8:07 am",100% Remote,$,NaN,Full-time,0-1 year,8 am,NaN,NaN


Now looking at the focus column - we have lots of focuses that are more than just one focus.

In [155]:
Counter(df['focus'])

Counter({'Frontend Developer': 82,
         'Backend Developer': 92,
         'Fullstack Developer': 189,
         'UX Designer': 114,
         'Data Analyst': 210,
         'Data Scientist': 100,
         'IT Support': 114,
         'Penetration Tester': 50,
         'Security Analyst': 155,
         'UI Designer': 56,
         'Outbound Sales Representative': 1,
         'UI Designer , UX Designer': 24,
         'Frontend Developer , Backend Developer': 2,
         'Backend Developer , Business Analyst': 1,
         'Fullstack Developer , Scrum': 26,
         'Fullstack Developer , Project Management': 2,
         'UX Designer , UX Researcher': 5,
         'UI Designer , Scrum': 4,
         'UI Designer , Frontend Developer': 1,
         'UI Designer , UX Researcher , UX Designer , Product Manager': 1,
         'UI Designer , Strategy': 1,
         'UI Designer , UX Designer , Product Manager': 3,
         'Data Analyst , Finance , Business Analyst': 1,
         'Data Analyst , Busin

Jobs with multiple focuses mostly show up only once, and if we take the first focus as the focus then this should balance out and not create any major bias just by looking at the above dictionary.

Se we want to put all focuses if there are multiple into a list in a new column. Then will iterate through them to replace the focus with the first focus.

In [156]:
#splitting the focus feature by commas so if there are more than one focus, 
#this will put them in a new column as a list, and remove white space
df['list_of_focus'] = df['focus'].str.split(',').apply(lambda x: [item.strip() for item in x])

df.head()

,company,description,focus,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience,postingHour,minSalary,maxSalary,list_of_focus
0,Novovu,[b]We are looking for a talented frontend web ...,Frontend Developer,"Frontend Developer (HTML, CSS, and JS) - Remote",https://lensa.com/frontend-developer-html-css-...,"Fort Washington, PA, USA","Jul 14, 2023 8:00 am",100% Remote,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,[Frontend Developer]
1,Coalition Technologies,[heading2]WHY YOU SHOULD APPLY:[/heading2]\r\n...,Frontend Developer,Front End Developer,https://www.virtualvocations.com/job/remote-fr...,NaN,"Jul 14, 2023 8:01 am",100% Remote,$,"[15, 35]",Full-time,0-1 year,8 am,15.0,35.0,[Frontend Developer]
2,Get It Recruit - Information Technology,[heading2]Job Description[/heading2]\r\n\r\nAs...,Backend Developer,Software Developer - Remote | WFH,https://www.linkedin.com/jobs/view/software-de...,"King George, VA 22485, USA","Jul 14, 2023 8:03 am",100% In-Office,$,"[72000, 182000]",Full-time,0-1 year,8 am,72000.0,182000.0,[Backend Developer]
3,SiLo,"[b]Based in Downtown Nashville, Simple Logisti...",Fullstack Developer,Junior .NET Full Stack Developer,https://jobs.wjhl.com/jobs/junior-.net-full-st...,"Nashville, TN, USA","Jul 14, 2023 8:04 am",100% In-Office,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,[Fullstack Developer]
4,eDiligence,[center][size=4][b]Entry Level Full Stack Deve...,Fullstack Developer,Entry Level Software Developer,https://jooble.org/jdp/-2571818786162603564/En...,"Los Angeles, CA, USA","Jul 14, 2023 8:05 am",Hybrid Work,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,[Fullstack Developer]


In [157]:
df['list_of_focus'] = df['list_of_focus'].apply(lambda x: x[0])

In [158]:
Counter(df['list_of_focus'])

Counter({'Frontend Developer': 84,
         'Backend Developer': 100,
         'Fullstack Developer': 229,
         'UX Designer': 131,
         'Data Analyst': 299,
         'Data Scientist': 148,
         'IT Support': 116,
         'Penetration Tester': 52,
         'Security Analyst': 172,
         'UI Designer': 92,
         'Outbound Sales Representative': 1,
         'Strategy': 1,
         'Finance': 1})

We have 3 one offs - looking at them above, Strategy and Finance have a second focus being IT support, so let's change it to that. Then, Outbound Sales doesn't have a second focus, but it is also not a tech job, so let's drop that row.

In [159]:
#getting the index
df[df['list_of_focus'] == 'Strategy']['list_of_focus']

726    Strategy
Name: list_of_focus, dtype: object

In [160]:
df[df['list_of_focus'] == 'Finance']['list_of_focus']

730    Finance
Name: list_of_focus, dtype: object

In [161]:
df.at[730, 'list_of_focus'] = 'IT Support'
df.at[726, 'list_of_focus'] = 'IT Support'

In [162]:
df[df['list_of_focus'] == 'Outbound Sales Representative']['list_of_focus']

86    Outbound Sales Representative
Name: list_of_focus, dtype: object

In [163]:
df.drop(86, inplace=True)

Double check list of focuses.

In [164]:
Counter(df['list_of_focus'])

Counter({'Frontend Developer': 84,
         'Backend Developer': 100,
         'Fullstack Developer': 229,
         'UX Designer': 131,
         'Data Analyst': 299,
         'Data Scientist': 148,
         'IT Support': 118,
         'Penetration Tester': 52,
         'Security Analyst': 172,
         'UI Designer': 92})

Now that it looks good, replace the focus column with the list of focus column

In [165]:
df.drop(columns = 'focus', inplace = True)

In [166]:
df.head()

,company,description,jobTitle,link,location,postedDate,remote,salaryCurrency,salaryRange,type,yearsOfExperience,postingHour,minSalary,maxSalary,list_of_focus
0,Novovu,[b]We are looking for a talented frontend web ...,"Frontend Developer (HTML, CSS, and JS) - Remote",https://lensa.com/frontend-developer-html-css-...,"Fort Washington, PA, USA","Jul 14, 2023 8:00 am",100% Remote,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,Frontend Developer
1,Coalition Technologies,[heading2]WHY YOU SHOULD APPLY:[/heading2]\r\n...,Front End Developer,https://www.virtualvocations.com/job/remote-fr...,NaN,"Jul 14, 2023 8:01 am",100% Remote,$,"[15, 35]",Full-time,0-1 year,8 am,15.0,35.0,Frontend Developer
2,Get It Recruit - Information Technology,[heading2]Job Description[/heading2]\r\n\r\nAs...,Software Developer - Remote | WFH,https://www.linkedin.com/jobs/view/software-de...,"King George, VA 22485, USA","Jul 14, 2023 8:03 am",100% In-Office,$,"[72000, 182000]",Full-time,0-1 year,8 am,72000.0,182000.0,Backend Developer
3,SiLo,"[b]Based in Downtown Nashville, Simple Logisti...",Junior .NET Full Stack Developer,https://jobs.wjhl.com/jobs/junior-.net-full-st...,"Nashville, TN, USA","Jul 14, 2023 8:04 am",100% In-Office,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,Fullstack Developer
4,eDiligence,[center][size=4][b]Entry Level Full Stack Deve...,Entry Level Software Developer,https://jooble.org/jdp/-2571818786162603564/En...,"Los Angeles, CA, USA","Jul 14, 2023 8:05 am",Hybrid Work,$,NaN,Full-time,0-1 year,8 am,NaN,NaN,Fullstack Developer


Now let's rename the list of focus column to just focus, but let's call it jobTitle actually, since that is really what it is. We need to remove that column now too, and let's remove the company, link, postedDate, location, salaryCurrency, and SalaryRange columns as well since we won't need them. 

In [167]:
df.drop(columns = ['description', 'company', 'link', 'jobTitle', 'location', 'postedDate', 'salaryCurrency', 'salaryRange'], inplace = True)

In [168]:
df.rename(columns={'list_of_focus': 'jobTitle'}, inplace=True)

In [169]:
df.head()

,remote,type,yearsOfExperience,postingHour,minSalary,maxSalary,jobTitle
0,100% Remote,Full-time,0-1 year,8 am,NaN,NaN,Frontend Developer
1,100% Remote,Full-time,0-1 year,8 am,15.0,35.0,Frontend Developer
2,100% In-Office,Full-time,0-1 year,8 am,72000.0,182000.0,Backend Developer
3,100% In-Office,Full-time,0-1 year,8 am,NaN,NaN,Fullstack Developer
4,Hybrid Work,Full-time,0-1 year,8 am,NaN,NaN,Fullstack Developer


## Description/Text Cleaning

Now we can handle the description part. 

In [170]:
sum(df.description.isnull())

AttributeError: 'DataFrame' object has no attribute 'description'

Only 4 null values, not much to worry about here.

To get the skills, probably want to do some tokenization with the descriptions, grouping by jobTitle - or previously known as the focus - and the most frequent token will hopefully be the technical skills.

In [ ]:
df.description[1]

grouping by job title

In [ ]:
#filling nulls in first so next operation works
df['description'].fillna('', inplace=True)

grouped_df = df.groupby('jobTitle')['description'].apply(lambda x: ' '.join(x)).reset_index()

In [ ]:
grouped_df

We have a lot of text to get rid of, as seen below. anything in brackets should be replaced by spaces, and then \r and \n should be removed as well.

In [ ]:
grouped_df.iloc[0]['description']

In [ ]:
#pattern to get rid of anything in brackets
pattern = r'\[.*?\]'

grouped_df['description'] = grouped_df['description'].apply(lambda x: re.sub(pattern, '', x))

In [ ]:
grouped_df.iloc[0]['description']

quickly scanning this it seems it is just punctuations and \r and \n that need to be removed now.

In [ ]:
grouped_df['description'] = grouped_df['description'].apply(lambda x: x.replace('\r', ' '))
grouped_df['description'] = grouped_df['description'].apply(lambda x: x.replace('\n', ' '))

Now just punctuations.

In [ ]:
punctuation_list = list(string.punctuation)

for punc in punctuation_list:
    grouped_df['description'] = grouped_df['description'].apply(lambda x: x.replace(punc, ' '))

In [ ]:
grouped_df.iloc[0]['description']

One more time, bullet points weren't in the punctuation list so we need to do that too.

In [ ]:
grouped_df['description'] = grouped_df['description'].apply(lambda x: x.replace('•', ' '))

In [ ]:
grouped_df.iloc[0]['description']

removing extra spaces

In [ ]:
grouped_df['description'] = grouped_df['description'].apply(lambda x: x.split())
grouped_df['description'] = grouped_df['description'].apply(lambda x: ' '.join(x))

grouped_df.iloc[0]['description']

In [ ]:
#let's also lower case everything now
grouped_df['description'] = grouped_df['description'].apply(lambda x: x.lower())

#grouped_df.iloc[0]['description']

Now let's remove stop words.

In [ ]:
nltk.download('stopwords')
#set of stopwords to search through
stop_words = set(stopwords.words('english'))

#tokenize and then remove stop words 
grouped_df['description'] = grouped_df['description'].apply(lambda x: nltk.word_tokenize(x))
grouped_df['description'] = grouped_df['description'].apply(lambda x: [word for word in x if word not in stop_words])

**Easiest thing to do here since hard technical skills aren't the most common is to look manually for the top 5 of each one, and note them down in another dictinary of dataframe.**

In [ ]:
#backend
Counter(grouped_df.iloc[0]['description']).most_common()

In [ ]:
backend_top_skills = {'java': 67, 'aws': 63, 'python': 59, 'react': 48, 'javascript': 47}

In [ ]:
#data analyst
Counter(grouped_df.iloc[1]['description']).most_common()

In [ ]:
data_analyst_top_skills = {'sql': 315, 'python': 219, 'excel': 193, 'tableau': 164, 'power': 114}

In [ ]:
#data scientist
Counter(grouped_df.iloc[2]['description']).most_common()

In [ ]:
data_scientist_top_skills = {'python': 141, 'sql': 121, 'r': 93, 'tableau': 49, 'aws': 32}

In [ ]:
#frontend
Counter(grouped_df.iloc[3]['description']).most_common()

In [ ]:
frontend_top_skills = {'javascript': 78, 'html': 60, 'react': 60, 'css': 53, 'sql': 28}

In [ ]:
#fullstack
Counter(grouped_df.iloc[4]['description']).most_common()

In [ ]:
fullstack_top_skills = {'javascript': 223, 'aws': 180, 'react': 165, 'sql': 155, 'java': 151}

In [ ]:
#IT support
Counter(grouped_df.iloc[5]['description']).most_common()

In [ ]:
it_support_top_skills = {'office': 95, 'windows': 75, 'ios': 15, 'jira': 9, 'linux': 8}

In [ ]:
#Penetration Tester
Counter(grouped_df.iloc[6]['description']).most_common()

In [ ]:
penetration_tester_top_skills = {'linux': 33, 'windows': 29, 'python': 27, 'java': 16, 'metasploit': 16}

In [ ]:
#Security Analyst
Counter(grouped_df.iloc[7]['description']).most_common()

In [ ]:
security_analyst_top_skills = {'windows': 58, 'siem': 56, 'soc': 56, 'azure': 32, 'linux': 26}

In [ ]:
#UI Designer
Counter(grouped_df.iloc[8]['description']).most_common()

In [ ]:
ui_top_skills = {'wireframes': 82, 'user flow': 78, 'prototypes': 61, 'figma': 45, 'javascript': 40}

In [ ]:
#UX Designer
Counter(grouped_df.iloc[9]['description']).most_common()

In [ ]:
ux_top_skills = {'prototypes': 87, 'user flow': 87, 'wireframes': 81, 'figma': 81, 'adobe': 60}

Now that we have all of these, let's add them to the grouped dataframe. Let;s get a list of all the lists created above. Then, lets put them in a dict to turn into a dataframe.

In [ ]:
job_skills_list = 
[
    backend_top_skills, 
    data_analyst_top_skills, 
    data_scientist_top_skills, 
    frontend_top_skills, 
    fullstack_top_skills, 
    it_support_top_skills, 
    penetration_tester_top_skills, 
    security_analyst_top_skills, 
    ui_top_skills, 
    ux_top_skills
]

In [ ]:
#using a default dict, we want to add lists for skills and frequencies, which we will fill in after.
skills_dict = defaultdict()

for i in range(5):
    skills_dict['skill_{}'.format(i+1)] = []
    skills_dict['freq_{}'.format(i+1)] = []

skills_dict

In [ ]:
#now let's fill in the skills and freq's by iterating over the job_skills_list and do it 5 times since we have 5 skills
for i in range(5):    
    for job in job_skills_list:
        skills_dict['skill_{}'.format(i+1)].append(list(job.items())[i][0])
        skills_dict['freq_{}'.format(i+1)].append(list(job.items())[i][1])

In [ ]:
#make it a dataframe
top_skills_df = pd.DataFrame(skills_dict)
top_skills_df

In [ ]:
#we don't need the descriptions anymore, so let's just get the jobTitles and concat these dataframes.
top_5_skills_df = pd.concat([grouped_df['jobTitle'], top_skills_df], axis = 1)
top_5_skills_df

And that's it. Now let's export this and the previous dataframe that isn't grouped as csv files so we can upload them to Tableau.

In [ ]:
top_5_skills_df.to_csv('top_5_skills_per_job.csv', index=False)
df.to_csv('cleaned_job_data.csv', index=False)

The following is just an idea for future improvement, and is not completed either.

In [ ]:
#if wanted to automate getting most common skills, could make a list of most common ones manually, and search for them
backend_skills = ['python', 'java', 'ruby', 'node', 'django', 'flask', 'sql', 'nosql', 'linux', 'aws', 'docker', 
                  'api', 'git', 'php', 'ruby', 'express', 'api', 'json', 'xml', 'github']

frontend_skills = ['html', 'css', 'javascript', 'react', 'angular', 'vue', 'ux', 'ui', 'redux', 'vuex', 'sass', 'less'
                'jquery', 'version', 'testing', 'unit', 'stylus', 'gui']

#fullstack is both of the above combined

#testing here is a/b testing, removing white space may have messed this word here. 
#google is also referring to google sheets and power is PowerBI
data_analyst_skills = ['python', 'r', 'tableau', 'power', 'matplotlib', 'ggplot2', 'sql', 'excel', 'google',
                       'statistics', 'statistical', 'cleaning', 'testing', 'visualization', 'business']

#Machine Learning, Deep Learning, Predictive Modeling are the full version of first three, and vision is computer vision
data_scientist_skills = ['machine', 'deep', 'predictive', 'spark', 'nlp', 'vision', 'ai', 'scikit', 'tensorflow', 
                         'pytorch', 'matlab', 'algebra', 'calculus']


it_support_skills = ['troubleshooting', 'operating', 'windows', 'macos', 'linux', 'network', 'cybersecurity', 
                     'desktop', 'aws', 'azure', 'gcp']

penetration_tester_skills = [
    "Ethical Hacking",
    "Network Scanning",
    "Vulnerability Assessment",
    "Exploitation Techniques",
    "Web Application Testing",
    "Wireless Network Security",
    "Security Tools (Metasploit, Wireshark, Nmap)",
    "Report Writing",
    "Compliance Standards (ISO 27001, NIST)",
    "Social Engineering"]

security_analyst_skills = penetration_tester_skills

ux_designer_skills = [
    "User Research",
    "Information Architecture",
    "Wireframing and Prototyping",
    "Usability Testing",
    "Interaction Design",
    "Persona Development",
    "User Flows",
    "Visual Design Principles",
    "Design Thinking",
    "UX/UI Tools (Sketch, Figma, Adobe XD)"
]

ui_designer_skills = [
    "Visual Design",
    "Graphic Design Software (Adobe Creative Suite)",
    "Color Theory",
    "Typography",
    "Icon Design",
    "User Interface Patterns",
    "Responsive Design",
    "Design Systems",
    "Animation",
    "Collaboration with Developers"
]
